In [1]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm

from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


os.environ['PATH'] += r'C:\Users\LENOVO\AppData\Local\selenium\chromedriver-win64'

options = Options()
# options.add_experimental_option("detach", True)
options.add_argument("--headless=new")


**match summary scraping to csv**

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
    "Content-Type": "application/x-www-form-urlencoded"}
try:
    html_text = requests.get('https://www.espncricinfo.com/records/tournament/team-match-results/icc-men-s-t20-world-cup-2022-23-14450',headers=headers)
    html_text.raise_for_status()
except Exception as e:
    print(e)
    print('url not found')
soup = BeautifulSoup(html_text.text, 'lxml')

In [3]:
full_data = soup.find('tbody').find_all('tr')

In [4]:
full_list = []
match_id_link = {}
for rows in full_data:
    each_dict = {}
    each_value = rows.find_all('td')

    
    each_dict['team1'] = each_value[0].text.strip()
    each_dict['team2'] = each_value[1].text.strip()
    each_dict['winner'] = each_value[2].text.strip()
    each_dict['margin'] = each_value[3].text.strip()
    each_dict['ground'] = each_value[4].text.strip()
    each_dict['match_date'] = each_value[5].text.strip()
    each_dict['match_id'] = each_value[6].text.strip()

    espn = 'https://www.espncricinfo.com'

    match_id_link[each_value[6].text.strip()] = [espn + each_value[6].a['href'], each_dict['team1'] + ' Vs ' +  each_dict['team2']]

    full_list.append(each_dict)


In [5]:
# match_id_link

{'T20I # 1823': ['https://www.espncricinfo.com/series/icc-men-s-t20-world-cup-2022-23-1298134/namibia-vs-sri-lanka-1st-match-first-round-group-a-1298135/full-scorecard',
  'Namibia Vs Sri Lanka'],
 'T20I # 1825': ['https://www.espncricinfo.com/series/icc-men-s-t20-world-cup-2022-23-1298134/netherlands-vs-united-arab-emirates-2nd-match-first-round-group-a-1298136/full-scorecard',
  'Netherlands Vs U.A.E.'],
 'T20I # 1826': ['https://www.espncricinfo.com/series/icc-men-s-t20-world-cup-2022-23-1298134/scotland-vs-west-indies-3rd-match-first-round-group-b-1298137/full-scorecard',
  'Scotland Vs West Indies'],
 'T20I # 1828': ['https://www.espncricinfo.com/series/icc-men-s-t20-world-cup-2022-23-1298134/ireland-vs-zimbabwe-4th-match-first-round-group-b-1298138/full-scorecard',
  'Ireland Vs Zimbabwe'],
 'T20I # 1830': ['https://www.espncricinfo.com/series/icc-men-s-t20-world-cup-2022-23-1298134/namibia-vs-netherlands-5th-match-first-round-group-a-1298139/full-scorecard',
  'Namibia Vs Nether

In [6]:
match_summary_df = pd.DataFrame(full_list)
match_summary_df.head()

,team1,team2,winner,margin,ground,match_date,match_id
0,Namibia,Sri Lanka,Namibia,55 runs,Geelong,"Oct 16, 2022",T20I # 1823
1,Netherlands,U.A.E.,Netherlands,3 wickets,Geelong,"Oct 16, 2022",T20I # 1825
2,Scotland,West Indies,Scotland,42 runs,Hobart,"Oct 17, 2022",T20I # 1826
3,Ireland,Zimbabwe,Zimbabwe,31 runs,Hobart,"Oct 17, 2022",T20I # 1828
4,Namibia,Netherlands,Netherlands,5 wickets,Geelong,"Oct 18, 2022",T20I # 1830


In [7]:
# match_summary_df.to_csv('match_summary.csv', index=False)

**batting and bowling data scraping**

In [8]:
batting_list = []
bowling_list = []
for id in match_id_link:
    
    match_id = id
    match_url = match_id_link[id][0]
    # match_url = match_id_link['T20I # 1823'][0]
    
    match_html_text = requests.get(match_url,headers=headers).text
    match_soup = BeautifulSoup(match_html_text, 'lxml')
    
    innings_soup = match_soup.find_all('div', class_="ds-rounded-lg ds-mt-2")

    for innings in innings_soup:
        # batting scraping
        batting_position = 0
        batting_team = innings.find('span', class_="ds-text-title-xs ds-font-bold ds-text-typo").text
        
        batting_soup = innings.find('table', class_="ds-w-full ds-table ds-table-md ds-table-auto ci-scorecard-table").find('tbody')
        each_row = batting_soup.find_all('tr', class_="")

        for rows in each_row:
            batting_position = batting_position + 1
            each_value = rows.find_all('td')
            
            if (each_value[0].name == 'td') and (each_value[0].text == 'TOTAL'):
                break
            batting_dict = {}
            
            # batting columns
            batting_dict['match_id'] = match_id
            batting_dict['match'] = match_id_link[id][1]
            batting_dict['batting_team'] = batting_team.split('(')[0].strip()
            batting_dict['batting_position'] = batting_position
            batting_dict['batsman'] = each_value[0].text.split('\xa0')[0].strip()
            batting_dict['out/not_out'] = each_value[1].text.strip()
            batting_dict['runs'] = each_value[2].text.strip()
            batting_dict['balls'] = each_value[3].text.strip()
            batting_dict['4s'] = each_value[5].text.strip()
            batting_dict['6s'] = each_value[6].text.strip()
            batting_dict['strike_rate'] = each_value[7].text.strip()

            batting_list.append(batting_dict)

        # bowling scraping
        bowling_team_without_vs = batting_dict['match'].replace('Vs','')
        bowling_team = bowling_team_without_vs.replace(batting_dict['batting_team'],'')

        bowling_soup = innings.find('table', class_="ds-w-full ds-table ds-table-md ds-table-auto").find('tbody')
        b_each_row = bowling_soup.find_all('tr', class_="")

        for b_row in b_each_row:
            b_each_value = b_row.find_all('td')
            bowling_dict = {}

            bowling_dict['match_id'] = match_id
            bowling_dict['match'] = match_id_link[id][1]
            bowling_dict['bowling_team'] = bowling_team.strip() 
            bowling_dict['bowler'] = b_each_value[0].text.strip()
            bowling_dict['overs'] = b_each_value[1].text.strip()
            bowling_dict['maidens'] = b_each_value[2].text.strip()
            bowling_dict['runs'] = b_each_value[3].text.strip()
            bowling_dict['wicket'] = b_each_value[4].text.strip()
            bowling_dict['economy'] = b_each_value[5].text.strip()
            bowling_dict['0s'] = b_each_value[6].text.strip()
            bowling_dict['4s'] = b_each_value[7].text.strip()
            bowling_dict['6s'] = b_each_value[8].text.strip()
            bowling_dict['wide'] = b_each_value[9].text.strip()
            bowling_dict['no_ball'] = b_each_value[10].text.strip()

            bowling_list.append(bowling_dict)
            
    
        


In [9]:
batting_df = pd.DataFrame(batting_list)
batting_df.tail()

,match_id,match,batting_team,batting_position,batsman,out/not_out,runs,balls,4s,6s,strike_rate
694,T20I # 1879,England Vs Pakistan,England,3,Phil Salt,c Iftikhar Ahmed b Haris Rauf,10,9,2,0,111.11
695,T20I # 1879,England Vs Pakistan,England,4,Ben Stokes,not out,52,49,5,1,106.12
696,T20I # 1879,England Vs Pakistan,England,5,Harry Brook,c Shaheen Shah Afridi b Shadab Khan,20,23,1,0,86.95
697,T20I # 1879,England Vs Pakistan,England,6,Moeen Ali,b Mohammad Wasim,19,13,3,0,146.15
698,T20I # 1879,England Vs Pakistan,England,7,Liam Livingstone,not out,1,1,0,0,100.00


In [10]:
# batting_df.to_csv('batting_data.csv', index=False)

In [11]:
bowling_df = pd.DataFrame(bowling_list)
bowling_df.tail(10)

,match_id,match,bowling_team,bowler,overs,maidens,runs,wicket,economy,0s,4s,6s,wide,no_ball
490,T20I # 1879,England Vs Pakistan,England,Sam Curran,4,0,12,3,3.00,15,0,0,0,0
491,T20I # 1879,England Vs Pakistan,England,Adil Rashid,4,1,22,2,5.50,10,1,0,1,0
492,T20I # 1879,England Vs Pakistan,England,Chris Jordan,4,0,27,2,6.75,9,3,0,0,0
493,T20I # 1879,England Vs Pakistan,England,Liam Livingstone,1,0,16,0,16.00,1,1,1,1,0
494,T20I # 1879,England Vs Pakistan,Pakistan,Shaheen Shah Afridi,2.1,0,13,1,6.00,6,1,0,0,0
495,T20I # 1879,England Vs Pakistan,Pakistan,Naseem Shah,4,0,30,0,7.50,15,3,1,1,0
496,T20I # 1879,England Vs Pakistan,Pakistan,Haris Rauf,4,0,23,2,5.75,13,3,0,1,0
497,T20I # 1879,England Vs Pakistan,Pakistan,Shadab Khan,4,0,20,1,5.00,10,1,0,0,0
498,T20I # 1879,England Vs Pakistan,Pakistan,Mohammad Wasim,4,0,38,1,9.50,5,5,0,2,0
499,T20I # 1879,England Vs Pakistan,Pakistan,Iftikhar Ahmed,0.5,0,13,0,15.60,0,1,1,0,0


In [12]:
# bowling_df.to_csv('bowling_data.csv', index=False)

**over by over comparison**

In [13]:
match_id_over_link = {}
for id in match_id_link:
    match_id_over_link[id] = [match_id_link[id][0].replace('full-scorecard','match-overs-comparison'), match_id_link[id][1]]
# print(match_id_over_link)

In [14]:
'''i = 0
over_by_over = []
for id in match_id_over_link:
    if i > 0:
        break
    i += 1
    
    match_id = id
    match_url = match_id_over_link[id][0]

    match_html_text = requests.get(match_url,headers=headers).text
    match_soup = BeautifulSoup(match_html_text, 'lxml')

    overs_soup = match_soup.find('table', class_="ds-w-full ds-table ds-table-md ds-table-bordered ds-border-collapse ds-border ds-border-line ds-table-fixed")
    team_names = overs_soup.find('thead').find('tr').find_all('th')
    team1 = team_names[1].text.strip()
    team2 = team_names[2].text.strip()

    each_row = overs_soup.find('tbody').find_all('tr')
    k = 1
    for rows in each_row:
        if k > 2:
            break
        k = k + 1

        value = rows.find_all('td')
        print(value[1].prettify)
        if value[0]:
            over_num = value[0].text.strip()

        # team 1
        if value[1]:
            team1_over = {}
            runs_and_wickets = value[1].span.text
            bowler_and_balls_list = value[1].find_all('div', attrs = {"class":"ds-text-typo"})
            # bowler_and_balls_list = value[1].find_all(lambda tag: tag.name == 'div' and tag.get('class') == ['ds-text-typo'])
            # for i in bowler_and_balls_list:
            #     print(i.prettify())
            #     print('****')
            # print(bowler_and_balls_list)
            # parent_div = value[1].div
            # x = {}
            # y = 1
            # for child in parent_div.children:
            #     x[y] = child
            #     y = y + 1
            # print(x[2].find('div').text)
            # bowler_and_balls_list = value[1].div.find('div')
            # print(value[1].find('div', class_="ds-text-compact-xs ds-pt-2 ds-mt-2"))
            # print('*'*15)
            # bowler_and_balls = bowler_and_balls_list[1].find_all('div')
            
            # bowler = bowler_and_balls[0].text
            # print(bowler)
            # balls = [ball.text for ball in bowler_and_balls[1]]
            
            team1_over['match_id'] = match_id
            team1_over['match'] = match_id_over_link[id][1]
            team1_over['team'] = team1
            team1_over['over'] = over_num
            team1_over['runs'] = runs_and_wickets.split(',')[0].replace('(','').replace('runs','').strip()
            team1_over['wickets'] = runs_and_wickets.split(',')[1].replace(')','').replace('wkt','').strip()
            # team1_over['bowler'] = bowler.split(':')[1].strip()
            # team1_over['balls'] = balls
            
            over_by_over.append(team1_over)

        # team 2
        if value[2]:
            team2_over = {}
            runs_and_wickets = value[2].span.text
            # bowler_and_balls_list = value[2].find_all('div', class_="ds-text-typo")
            # bowler_and_balls = bowler_and_balls_list[1].find_all('div')
            # bowler_and_balls_list = value[2].div.find_all('div', recursive=False)
            # print(len(bowler_and_balls_list))
            # bowler = bowler_and_balls[0].text
            # balls = [ball.text for ball in bowler_and_balls[1]]
            
            team2_over['match_id'] = match_id
            team2_over['match'] = match_id_over_link[id][1]
            team2_over['team'] = team2
            team2_over['over'] = over_num
            team2_over['runs'] = runs_and_wickets.split(',')[0].replace('(','').replace('runs','').strip()
            team2_over['wickets'] = runs_and_wickets.split(',')[1].replace(')','').replace('wkt','').strip()
            # team2_over['bowler'] = bowler.split(':')[1].strip()
            # team2_over['balls'] = balls
    
            over_by_over.append(team2_over)
        # print(over_by_over)

'''

'i = 0\nover_by_over = []\nfor id in match_id_over_link:\n    if i > 0:\n        break\n    i += 1\n    \n    match_id = id\n    match_url = match_id_over_link[id][0]\n\n    match_html_text = requests.get(match_url,headers=headers).text\n    match_soup = BeautifulSoup(match_html_text, \'lxml\')\n\n    overs_soup = match_soup.find(\'table\', class_="ds-w-full ds-table ds-table-md ds-table-bordered ds-border-collapse ds-border ds-border-line ds-table-fixed")\n    team_names = overs_soup.find(\'thead\').find(\'tr\').find_all(\'th\')\n    team1 = team_names[1].text.strip()\n    team2 = team_names[2].text.strip()\n\n    each_row = overs_soup.find(\'tbody\').find_all(\'tr\')\n    k = 1\n    for rows in each_row:\n        if k > 2:\n            break\n        k = k + 1\n\n        value = rows.find_all(\'td\')\n        print(value[1].prettify)\n        if value[0]:\n            over_num = value[0].text.strip()\n\n        # team 1\n        if value[1]:\n            team1_over = {}\n            

In [23]:
driver = Chrome(options=options)

over_by_over = []

for id in tqdm(match_id_over_link, desc="Processing", unit="matches", total=len(match_id_over_link)):
    match_id = id
    match_url = match_id_over_link[id][0]

    driver.get(match_url)

    driver.implicitly_wait(5)
   
    try:
        no_thanks_button = WebDriverWait(driver,15).until(
            EC.element_to_be_clickable((By.ID,'wzrk-cancel'))
        )
        no_thanks_button.click()
    except:
        print("Pop up didn't appear for ",match_id_over_link[id][1])

    wait = WebDriverWait(driver,5)

    click_elements = driver.find_elements(By.XPATH,'//td[@class="ds-min-w-max ds-cursor-pointer ds-align-top"]')
    for element in click_elements[1:]:
        element.click()
        new = element.find_element(By.XPATH, './following-sibling::td[1]//div[@class="ds-text-compact-xs ds-pt-2 ds-mt-2"]')
        wait.until(EC.visibility_of(new))

    html = driver.page_source
    match_soup = BeautifulSoup(html,'lxml')

    overs_soup = match_soup.find('table', class_="ds-w-full ds-table ds-table-md ds-table-bordered ds-border-collapse ds-border ds-border-line ds-table-fixed")
    team_names = overs_soup.find('thead').find('tr').find_all('th')
    team1 = team_names[1].text.strip()
    team2 = team_names[2].text.strip()

    each_row = overs_soup.find('tbody').find_all('tr')


    for rows in each_row:

        value = rows.find_all('td')

        if value[0].div:
            over_num = value[0].text.strip()

        # team 1
        if value[1].span:
            team1_over = {}
            runs_and_wickets = value[1].span.text
            bowler_and_balls_list = value[1].find_all('div', attrs={"class": "ds-text-typo"})
            bowler_and_balls = bowler_and_balls_list[1].find_all('div')
            bowler = bowler_and_balls[0].text
            balls = [ball.text for ball in bowler_and_balls[1]]

            team1_over['match_id'] = match_id
            team1_over['match'] = match_id_over_link[id][1]
            team1_over['team'] = team2
            team1_over['over'] = over_num
            team1_over['runs'] = runs_and_wickets.split(',')[0].replace('(', '').replace('runs', '').strip()
            team1_over['wickets'] = runs_and_wickets.split(',')[1].replace(')', '').replace('wkt', '').strip()
            team1_over['bowler'] = bowler.split(':')[1].strip()
            team1_over['balls'] = balls

            over_by_over.append(team1_over)

        # team 2
        if value[2].span:
            team2_over = {}
            runs_and_wickets = value[2].span.text
            bowler_and_balls_list = value[2].find_all('div', class_="ds-text-typo")
            bowler_and_balls = bowler_and_balls_list[1].find_all('div')
            bowler = bowler_and_balls[0].text
            balls = [ball.text for ball in bowler_and_balls[1]]

            team2_over['match_id'] = match_id
            team2_over['match'] = match_id_over_link[id][1]
            team2_over['team'] = team1
            team2_over['over'] = over_num
            team2_over['runs'] = runs_and_wickets.split(',')[0].replace('(', '').replace('runs', '').strip()
            team2_over['wickets'] = runs_and_wickets.split(',')[1].replace(')', '').replace('wkt', '').strip()
            team2_over['bowler'] = bowler.split(':')[1].strip()
            team2_over['balls'] = balls

            over_by_over.append(team2_over)

driver.quit()
df = pd.DataFrame(over_by_over)
df.to_csv('bowler_balls1.csv', encoding='utf-8-sig',index=False)
    


Processing:   2%|█▌                                                                | 1/42 [00:27<18:39, 27.30s/matches]

Pop up didn't appear for  Netherlands Vs U.A.E.


Processing:   5%|███▏                                                              | 2/42 [00:49<16:14, 24.36s/matches]

Pop up didn't appear for  Scotland Vs West Indies


Processing:   7%|████▋                                                             | 3/42 [01:11<15:10, 23.35s/matches]

...

Processing:  95%|█████████████████████████████████████████████████████████████▉   | 40/42 [14:46<00:43, 21.79s/matches]

Pop up didn't appear for  England Vs India


Processing:  98%|███████████████████████████████████████████████████████████████▍ | 41/42 [15:08<00:21, 21.90s/matches]

Pop up didn't appear for  England Vs Pakistan


Processing: 100%|█████████████████████████████████████████████████████████████████| 42/42 [15:30<00:00, 22.16s/matches]


,match_id,match,team,over,runs,wickets,bowler,balls
0,T20I # 1823,Namibia Vs Sri Lanka,Sri Lanka,1,5,0,Maheesh Theekshana,"[•, •, 1, 1, 2, 1w, •]"
1,T20I # 1823,Namibia Vs Sri Lanka,Namibia,1,8,0,Gerhard Erasmus,"[4, 1, 1, •, 1, 1]"
2,T20I # 1823,Namibia Vs Sri Lanka,Sri Lanka,2,8,1,Dushmantha Chameera,"[1, W, 1, 4, •, 2]"
3,T20I # 1823,Namibia Vs Sri Lanka,Namibia,2,5,1,David Wiese,"[•, 2, 2, W, 1, •]"
4,T20I # 1823,Namibia Vs Sri Lanka,Sri Lanka,3,3,1,Pramod Madushan,"[1, 1, 1, •, •, W]"
...,...,...,...,...,...,...,...,...
1591,T20I # 1879,England Vs Pakistan,England,18,5,1,Chris Jordan,"[1, W, •, 3, 1, •]"
1592,T20I # 1879,England Vs Pakistan,Pakistan,18,5,0,Haris Rauf,"[1, 1, 2, •, •, 1]"
1593,T20I # 1879,England Vs Pakistan,England,19,4,1,Sam Curran,"[•, 2, W, 1, •, 1]"
1594,T20I # 1879,England Vs Pakistan,Pakistan,19,7,1,Mohammad Wasim,"[1, W, 1, 4, •, 1]"


AttributeError: 'function' object has no attribute 'sum'